In [1]:
import pandas as pd
import plotly.express as px


def _resolve_all_estimators():
    try:
        from sktime.registry import all_estimators
        return all_estimators
    except Exception:
        from sktime.utils.discovery import all_estimators
        return all_estimators


try:
    import sktime  # noqa: F401
    all_estimators = _resolve_all_estimators()
    SKTIME_AVAILABLE = True
except Exception as e:
    SKTIME_AVAILABLE = False
    _err = e

if not SKTIME_AVAILABLE:
    print("sktime not installed. Install with `pip install sktime` to populate the catalog.")
    print("Error:", type(_err).__name__, _err)
else:
    estimators = all_estimators()
    print(f"Total estimators found: {len(estimators)}")

sktime not installed. Install with `pip install sktime` to populate the catalog.
Error: ModuleNotFoundError No module named 'sktime'


In [2]:
if SKTIME_AVAILABLE:
    rows = []
    for name, cls in estimators:
        tags = {}
        if hasattr(cls, "get_class_tags"):
            try:
                tags = cls.get_class_tags()
            except Exception:
                tags = {}

        scitype = tags.get("scitype") or tags.get("estimator_type")
        task = tags.get("task") or tags.get("learning_type")

        rows.append(
            {
                "name": name,
                "class": cls.__name__,
                "module": cls.__module__,
                "module_family": ".".join(cls.__module__.split(".")[:2]),
                "scitype": scitype,
                "task": task,
                "multivariate": tags.get("capability:multivariate"),
                "missing_values": tags.get("capability:missing_values"),
                "unequal_length": tags.get("capability:unequal_length"),
            }
        )

    df = pd.DataFrame(rows).sort_values(["scitype", "name"])        .reset_index(drop=True)
    df.head(10)

In [3]:
if SKTIME_AVAILABLE:
    counts = (
        df["scitype"]
        .fillna("unknown")
        .value_counts()
        .reset_index()
        .rename(columns={"index": "scitype", "scitype": "count"})
    )
    fig = px.bar(
        counts,
        x="scitype",
        y="count",
        title="Estimator counts by scitype",
    )
    fig.show()

In [4]:
if SKTIME_AVAILABLE:
    fig = px.treemap(
        df,
        path=["scitype", "module_family"],
        title="Estimator distribution by module family",
    )
    fig.show()

In [5]:
if SKTIME_AVAILABLE:
    df